In [ ]:
%run VQAs_packages.ipynb

In [ ]:
targetDOF = 8192
%run FEM_lumped_system_for_PlaneStrain.ipynb

Total Free DOF is 8192, prescribed DOF is 44


In [ ]:
H = np.linalg.inv(M)*K

Test_A = H/np.linalg.norm(H)
print('condition number of normalized Hamiltonian: ', la.cond(Test_A))
print('L2 norm of normalized Hamiltonian: ', np.linalg.norm(Test_A))

qubitOp = MatrixOp(primitive=Test_A)
qubits = qubitOp.num_qubits
print(f"The Hamiltonian is a {2**qubits}x{2**qubits} matrix for {qubits} qubits with a total of {2**qubits} DOF")

condition number of normalized Hamiltonian:  137.62684683415335
L2 norm of normalized Hamiltonian:  1.0
The Hamiltonian is a 8192x8192 matrix for 13 qubits with a total of 8192 DOF


In [ ]:
npme = NumPyMinimumEigensolver()
result = npme.compute_minimum_eigenvalue(operator=qubitOp)
ref_value = result.eigenvalue.real
print(f'Reference value: {ref_value:.10f}')

Reference value: 0.0003187112


In [ ]:
# Convergence of VQE VQESolver with a set of optimizers

start_time = time.time()
warnings.filterwarnings("ignore")

rngseed = 0
max_iteration = 1e5
tolerance = 1e-4
shots = 1e5
optimizer = COBYLA(maxiter=max_iteration)

# var_form_depths = [3, 6, 10]
var_form_depths = [10]
converge_cnts = np.empty([len(var_form_depths)], dtype=object)
converge_vals = np.empty([len(var_form_depths)], dtype=object)
vqe_result = np.empty([len(var_form_depths)], dtype=object)

mode = 'Layers'
name = f'VQE_Convergence_PlaneStrain_lumped_{qubits}Q_{mode}{var_form_depths[0]}_data.txt'
path_convergence_layers = r'C:\\temp\\' + str(name)

with open(path_convergence_layers, 'a') as f:
    f.write("{1}{0}".format(str(qubits), 'Qubits Number: '))
    f.write('\n')

for i, var_form_depth in enumerate(var_form_depths):
    print('\r var_form_depth: {}        '.format(str(var_form_depth)), end='')
    algorithm_globals.random_seed = rngseed
    var_form = TwoLocal(qubitOp.num_qubits, ['rz', 'ry'], 'cz', entanglement = 'full', reps = var_form_depth)
    backend = Aer.get_backend('statevector_simulator', max_parallel_threads=6, max_parallel_experiments=0)    
    q_instance = QuantumInstance(backend, shots=shots, seed_transpiler=rngseed, seed_simulator=rngseed)
    
    counts = []
    values = []
    def store_intermediate_result(eval_count, parameters, mean, std):
        counts.append(eval_count)
        values.append(mean)

    vqe = VQE(ansatz=var_form, optimizer=optimizer, quantum_instance=q_instance, callback=store_intermediate_result)
    result = vqe.compute_minimum_eigenvalue(operator=qubitOp)
    vqe_result[i] = np.real(result.eigenvalue)
    print(vqe_result[i])
    converge_cnts[i] = np.asarray(counts)
    converge_vals[i] = np.asarray(values)
    with open(path_convergence_layers, 'a') as f:
        f.write("{1}{0}".format(str(var_form_depth), 'var_form_depth: '))
        f.write('\n')
        f.write("{1}{0}".format(str(vqe_result[i]), 'vqe_result: '))
    print('\rOptimization complete      ')

end_time = time.time()
runtime = end_time-start_time
print('runtime (s):', runtime)


 var_form_depth: 10        0.000356784234567499
Optimization complete      
runtime (s): 50096.21796798706


In [ ]:
# Save Convergence Data to Excel

np.set_printoptions(threshold=sys.maxsize)

max_len = [len(converge_cnts[i]) for i in range(len(var_form_depths))]
p = max(max_len)
print(p)

A = converge_cnts[0]
A_pad = np.pad(A, (0, p - len(A)), constant_values=0)
data1 = np.hstack((A_pad))

B = converge_vals[0]
B_pad = np.pad(B, (0, p - len(B)), constant_values=min(B))
data2 = np.hstack((B_pad))
                   
data = np.hstack((data1, data2))

excel_name = f'VQE_Convergence_PlaneStrain_lumped_{qubits}Q_{mode}{var_form_depths[0]}_data.xlsx'
path_to_excel = r'C:\\temp\\' + str(excel_name)

ix = np.empty(len(converge_cnts[0]), dtype=object)
df = np.empty(len(converge_vals[0]), dtype=object)

dict_columns = dict( {'A': data[0:p-1], 'B': data[p:2*p-1]})
data = pd.DataFrame(dict_columns)
data.to_excel(path_to_excel, sheet_name='Sheet1', index=False, header=True)

100000


In [ ]:
# Find Minimum of converged data, get error compared to ref_value

file = pd.read_excel(path_to_excel)

Min = np.array([min(file['B'])])
errors = [(val - ref_value)*100/ref_value for val in Min]
datas = np.hstack((ref_value, errors))
print(errors)

error_name = f'VQE_Convergence_PlaneStrain_lumped_{qubits}Q_{mode}{var_form_depths[0]}_data_error.xlsx'
path_to_errors = r'C:\\temp\\' + str(error_name)

dict_errors = dict({'A': datas})
errors_fig = pd.DataFrame(dict_errors)
errors_fig.to_excel(path_to_errors, sheet_name='Sheet1', index=False, header=True)

In [ ]:
# Direct plot from data stored in Python

pylab.rcParams['figure.figsize'] = (10,8)
fig = plt.figure(dpi=600)
ax = fig.add_subplot(2, 1, 1)

for i, var_form_depth in enumerate(var_form_depths):
    pylab.plot(converge_cnts[i], converge_vals[i], label='depth: '+str(var_form_depth))
pylab.yscale('log')

xlimit = 4000
pylab.xlim(0, xlimit)

csfont = {'fontname':'Times New Roman'}
pylab.title('Classical Optimizer: COBYLA, Entanglement Pattern: CZ', **csfont, size=18)
pylab.xlabel('iteration times', **csfont, size=18)
pylab.ylabel('eigenvalue estimate', **csfont, size=18)

ticks_font = fm.FontProperties(
    family='Times New Roman', style='normal', size=16, weight='normal', stretch='normal')
for label in pylab.gca().get_xticklabels (): # get current axes
    label.set_fontproperties(ticks_font)
for label in pylab.gca().get_yticklabels ():
    label.set_fontproperties(ticks_font)
pylab.gca().yaxis.set_ticklabels([])

props = dict({'size': 16, 'family': 'Times New Roman', 'weight': 'normal', 'style': 'normal'})
pylab.legend(loc='best', prop=props)

pylab.show()

In [ ]:
# Draw data from Excel and plot

pylab.rcParams['figure.figsize'] = (10,8)
fig = plt.figure(dpi=600)
ax = fig.add_subplot(2, 1, 2)

file = pd.read_excel(path_to_excel)
x = np.array([file['A'], file['B'], file['C']])
y = np.array([file['D'], file['E'], file['F']])

for i, var_form_depth in enumerate(var_form_depths):
    pylab.plot(x[i], y[i], label='depth: '+str(var_form_depth))
# pylab.xscale('log')
pylab.yscale('log')

xlimit = 4000
pylab.xlim(0, xlimit)

csfont = {'fontname':'Times New Roman'}
pylab.title('Classical Optimizer: COBYLA, Entanglement Pattern: CZ', **csfont, size=12)
pylab.xlabel('iteration times', **csfont, size=12)
pylab.ylabel('eigenvalue estimate', **csfont, size=12)

ticks_font = fm.FontProperties(
    family='Times New Roman', style='normal', size=10, weight='normal', stretch='normal')
for label in pylab.gca().get_xticklabels (): # get current axes
    label.set_fontproperties(ticks_font)
for label in pylab.gca().get_yticklabels ():
    label.set_fontproperties(ticks_font)
    
pylab.gca().yaxis.set_ticklabels([])

props = dict({'size': 10, 'family': 'Times New Roman', 'weight': 'normal', 'style': 'normal'})
pylab.legend(loc='best', prop=props)

path_save_fig = 'C:/temp/fig.png'
pylab.savefig (path_save_fig)

pylab.show()

In [ ]:
# Monitor via logging

import logging
logging.basicConfig(level=logging.INFO) # set the logging level to INFO
logging.getLogger('qiskit.algorithms.minimum_eigen_solvers.vqe').setLevel(logging.INFO)

# This will generate detailed log files